In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import contextily as cx
import math
import random

# Make a map with occurrences of a list of species
def overlay_shapefile_on_raster(raster_path, rivers_path, major_rivers_path, lakes_path, borders_path):
    
    # Set the bounding box coordinates
    xmin, xmax, ymin, ymax = -69.6462, -57.4987, -22.8721, -9.6695
    
    # Open the raster file
    with rasterio.open(raster_path) as src:
        
        # Define the window using the provided bounding box
        window = rasterio.windows.from_bounds(xmin, ymin, xmax, ymax, transform=src.transform)
        # Read the subset of the raster data for Bolivia
        subset = src.read(window=window)
        # Log-transform the raster values (add 1 to avoid log(0)) to highlight altitude differences in the lowlands
        log_transformed_subset = np.log1p(subset)
        # Plot the log-transformed subset
        fig, ax = plt.subplots(figsize=(10, 10))
        ax = rasterio.plot.show(log_transformed_subset, transform=src.window_transform(window), ax=ax, cmap='terrain')
        
        # Open the shapefile with river network using geopandas
        gdf_rivers = gpd.read_file(rivers_path)
        gdf_rivers = gdf_rivers.cx[xmin:xmax, ymin:ymax]
        # Overlay the first shapefile on top of the raster map
        gdf_rivers.plot(ax=plt.gca(), facecolor='none', edgecolor='blue', linewidth=0.1)
        
        # Open the major river shapefile using geopandas
        gdf_major = gpd.read_file(major_rivers_path)
        gdf_major = gdf_major.cx[xmin:xmax, ymin:ymax]
        # Overlay the second shapefile on top of the raster map
        gdf_major.plot(ax=plt.gca(), facecolor='none', edgecolor='blue', linewidth=0.2)

        # Open the lakes shapefile using geopandas
        gdf_lakes = gpd.read_file(lakes_path)
        gdf_lakes = gdf_lakes.cx[xmin:xmax, ymin:ymax]
        # Overlay the second shapefile on top of the raster map
        gdf_lakes.plot(ax=plt.gca(), facecolor='blue', edgecolor='none')

        # Open the borders shapefile using geopandas
        gdf_borders = gpd.read_file(borders_path)
        gdf_borders = gdf_borders.cx[xmin:xmax, ymin:ymax]        
        # Overlay the second shapefile on top of the raster map
        gdf_borders.plot(ax=plt.gca(), facecolor='none', edgecolor='black', linewidth=0.5)
        
        # Save the map as a pdf
        plt.savefig(map, format='jpg', dpi=300)

        # Show the plot
        plt.show()

In [ ]:
names = ['Pinirampus pirinampu', 'Arapaima gigas', 'Cichla pleiozona']

raster_path = 'C:/Users/pascalh/Downloads/bolivia.tif'
rivers_path = 'C:/Users/pascalh/Downloads/rivers_samerica_37330(1)/rivers_samerica_37330.shp'
borders_path = 'C:/Users/pascalh/Documents/Projects/git_projects/Identifying_Bolivian_fishes/maps/BOL_adm/BOL_adm0.shp'
lakes_path = 'C:/Users/pascalh/Downloads/lagos_lagunas/lagos_lagunas.shp'
major_rivers_path = 'C:/Users/pascalh/Downloads/rios_mayores/rios_mayores.shp'
amazon_fish_path = 'C:/Users/pascalh/Downloads/CompleteDatabase2022_v2.csv'
gbif_path = 'C:/Users/pascalh/Documents/Projects/git_projects/Identifying_Bolivian_fishes/GBIF_20231101/occurrence.txt'
own_collection_path = 'C:/Users/pascalh/Documents/Projects/git_projects/Identifying_Bolivian_fishes/data/catalog own collection_decimal coordinates.csv'

overlay_shapefile_on_raster(raster_path, rivers_path, major_rivers_path, lakes_path, borders_path,
                                amazon_fish_path, own_collection_path, gbif_path, names)